# Phase de Nettoyage des données

Les différentes données récupérées ont tout d'abord besoin d'être nettoyées pour les besoins du projet. Si les données fournies par l'EUDA sont déjà très propres et ne nécéssiteront qu'un léger travail de tri, voire éventuellement quelques merge utiles sur certaines tables, ce n'est pas le cas des données scrapées à partir de pages Wikipedia qui vont nous demander un travail de nettoyage plus approfondi.

## Nettoyage des données scrapées depuis Wikipedia

On commence par générer notre dataframe à partir du scraping des données de [l'article Wikipedia sur la légalisation du Cannabis par Pays](https://fr.wikipedia.org/wiki/L%C3%A9gislation_sur_le_cannabis). Le code détaillé et commenté se trouve dans le fichier `scraping.py`, on se contente ici d'appeler nos fonctions.


In [1]:
from Data.scraping import scraping as s

url = "https://fr.wikipedia.org/wiki/L%C3%A9gislation_sur_le_cannabis"

table = s.scrape_first_table(url)
data, header = s.table_to_list(table)
legal_pays = s.make_pandas(data, header)

legal_pays.head()

{'Albanie': ['Illégale', 'Illégale', 'Illégal', 'Illégale', '[13]']}
{'Albanie': ['Illégale', 'Illégale', 'Illégal', 'Illégale', '[13]'], 'Algérie': ['Illégale', 'Illégale', 'Illégal', 'Illégale', 'Des autorisations pour usages médicaux et de recherche scientifique sont possibles.Article détaillé\xa0: Cannabis en Algérie.']}
{'Albanie': ['Illégale', 'Illégale', 'Illégal', 'Illégale', '[13]'], 'Algérie': ['Illégale', 'Illégale', 'Illégal', 'Illégale', 'Des autorisations pour usages médicaux et de recherche scientifique sont possibles.Article détaillé\xa0: Cannabis en Algérie.'], 'Allemagne': ["Légale jusqu'à 25 grammes", 'Légale pour les associations agréés', 'Légale', "Légale jusqu'à trois plants par personnes", "Article détaillé\xa0: Cannabis en Allemagne. Il existe un mythe selon lequel la possession de petites quantités de haschich ou de marijuana a été légalisée par le Tribunal constitutionnel fédéral en 1994. Ce n'est pas vrai. Le tribunal a seulement fait remarquer que d'éventuel

,Possession,Vente,Transport,Culture,Notes
Albanie,Illégale,Illégale,Illégal,Illégale,[13]
Algérie,Illégale,Illégale,Illégal,Illégale,Des autorisations pour usages médicaux et de r...
Allemagne,Légale jusqu'à 25 grammes,Légale pour les associations agréés,Légale,Légale jusqu'à trois plants par personnes,Article détaillé : Cannabis en Allemagne. Il e...
Arabie saoudite,Illégale,,,,Article détaillé : Cannabis en Arabie saoudite.
Argentine,Illégale,Illégale,Illégal,Illégale sauf à des fins médicinales depuis le...,Article détaillé : Cannabis en Argentine.


On remarque alors que notre Dataframe contient plusieurs éléments qui nécéssitent d'être nettoyés, modifiés, voire supprimés. Dans l'ordre on peut citer:

1. **Supprimer la dernier colonne concernant les "Notes"**, qui peuvent nous être utile à des fins d'interprétation, ou encore pour aider à la classification ou à l'interprétation plus tard, mais qui en l'état seront inutiles à la statistique descriptive ou à la construction de modèles plus tard.

2. **Eliminer de notre dataframe les pays non-européens**, puisque les données de l'EUDA ne s'intéresse qu'au cas européeen, et que c'est là le coeur de notre sujet.

3. Enfin, il nous faudra **regrouper les différentes états des variables sous un format clair** En l'état, on dispose pour chaque pays d'une description parfois unique et idividuelle de la législation dans le pays donné. L'idéal serait de ne garder qu'une variable binaire: Légal/Illégal. Cependant la réalité est plus complexe dans certains pays, et il nous faudra alors faire des choix, et les justifier, pour essayer de ne garder qu'un nombre limité d'état pour chaque variable.

## Suppression de la colonne des Notes

Cela n'est pas bien compliqué, on utilise simplement les outils de la librairie Pandas:

In [2]:
import pandas as pd

legal_pays_clean = legal_pays.drop(columns = "Notes")

legal_pays_clean.head()

,Possession,Vente,Transport,Culture
Albanie,Illégale,Illégale,Illégal,Illégale
Algérie,Illégale,Illégale,Illégal,Illégale
Allemagne,Légale jusqu'à 25 grammes,Légale pour les associations agréés,Légale,Légale jusqu'à trois plants par personnes
Arabie saoudite,Illégale,,,
Argentine,Illégale,Illégale,Illégal,Illégale sauf à des fins médicinales depuis le...


## Tri pour ne garder que les pays Européens

Cette opération n'est elle aussi pas très compliquée, on va se servir d'une table de correspondance scrapée sur le [site national de Statistiques Candiennes](https://www23.statcan.gc.ca/imdb/p3VD.pl?Function=getVD&TVD=1312110) (ce afin d'avoir le nom des pays en anglais), qui nous sera d'ailleurs aussi utile plus tard afin de faire correspondre chaque pays de l'Union européenne avec son code ISO 3316.

In [3]:
from Data.scraping import scraping as s

url = "https://www23.statcan.gc.ca/imdb/p3VD.pl?Function=getVD&TVD=1312110"

table = s.scrape_first_table(url)
data, header = s.table_to_list(table)
table_corres = s.make_pandas(data)

table_corres.head()

{'Belgium': ['BE', 'BEL']}
{'Belgium': ['BE', 'BEL'], 'Bulgaria': ['BG', 'BGR']}
{'Belgium': ['BE', 'BEL'], 'Bulgaria': ['BG', 'BGR'], 'Croatia': ['HR', 'HRV']}
{'Belgium': ['BE', 'BEL'], 'Bulgaria': ['BG', 'BGR'], 'Croatia': ['HR', 'HRV'], 'Cyprus': ['CY', 'CYP']}
{'Belgium': ['BE', 'BEL'], 'Bulgaria': ['BG', 'BGR'], 'Croatia': ['HR', 'HRV'], 'Cyprus': ['CY', 'CYP'], 'Czechia': ['CZ', 'CZE']}
{'Belgium': ['BE', 'BEL'], 'Bulgaria': ['BG', 'BGR'], 'Croatia': ['HR', 'HRV'], 'Cyprus': ['CY', 'CYP'], 'Czechia': ['CZ', 'CZE'], 'Denmark': ['DK', 'DNK']}
{'Belgium': ['BE', 'BEL'], 'Bulgaria': ['BG', 'BGR'], 'Croatia': ['HR', 'HRV'], 'Cyprus': ['CY', 'CYP'], 'Czechia': ['CZ', 'CZE'], 'Denmark': ['DK', 'DNK'], 'Estonia': ['EE', 'EST']}
{'Belgium': ['BE', 'BEL'], 'Bulgaria': ['BG', 'BGR'], 'Croatia': ['HR', 'HRV'], 'Cyprus': ['CY', 'CYP'], 'Czechia': ['CZ', 'CZE'], 'Denmark': ['DK', 'DNK'], 'Estonia': ['EE', 'EST'], 'Finland': ['FI', 'FIN']}
{'Belgium': ['BE', 'BEL'], 'Bulgaria': ['BG', 'BGR'], 

,0,1
Belgium,BE,BEL
Bulgaria,BG,BGR
Croatia,HR,HRV
Cyprus,CY,CYP
Czechia,CZ,CZE


On doit néanmoins ajouter à la main certains pays non-membres de l'Union Européenne, mais présent dans les données que nous avons à disposition (souvent des pays candidats comme la Turquie)

In [6]:
import pandas as pd

table_corres.loc["Austria"] = ["AT","AUT"]
table_corres.loc["Türkiye"] = ["TR","TUR"]
table_corres.loc["Norway"] = ["NO","NOR"]

table_corres.tail()

,0,1
Spain,ES,ESP
Sweden,SE,SWE
Austria,AT,AUT
Türkiye,TR,TUR
Norway,NO,NOR


# Ajout de nouvelles tables

**Tables de données sur les "entrants" pour des soins liés à la consommation de cannabis**

Définition :
**Entrants** : Personnes ayant entamé au cours de l'année un parcours de soin lié à un consommation de drogue (de cannabis ici).

On commence donc par "nettoyer" les tables principales en ne gardant à chaque fois que les colonnes qui nous intéressent :
- Pays
- Année
- Total d'entrants
- Nombre d'entrants liés au cannabis

In [ ]:
import pandas as pd
import openpyxl

### On commence par les donnees generales
total = "/home/onyxia/work/Projet_Python_2A/Data/EUDA/Cannabis_detaille/drogues_entrants.xlsx"
df1 = pd.read_excel(total, header=3)

# On ne garde que les colonnes qui nous intéressent
colonnes = ['Country', 'Year of Treatment', 'All substances', 'Cannabis']
df1 = df1[colonnes]


### On procede de la meme maniere pour les donnees concernant les personnes deja traitees auparavant
deja = "/home/onyxia/work/Projet_Python_2A/Data/EUDA/Cannabis_detaille/deja_traite_entrants.xlsx"
df2 = pd.read_excel(deja, header=3)

# On ne garde que les colonnes qui nous intéressent
colonnes = ['Country', 'Year of Treatment', 'All substances', 'Cannabis']
df2 = df2[colonnes]


### On procede de la meme maniere pour les donnees concernant les personnes jamais traitees auparavant
jamais = "/home/onyxia/work/Projet_Python_2A/Data/EUDA/Cannabis_detaille/jamais_traite_entrants.xlsx"
df3 = pd.read_excel(jamais, header=3)

# On ne garde que les colonnes qui nous intéressent
colonnes = ['Country', 'Year of Treatment', 'All substances', 'Cannabis']
df3 = df3[colonnes]


/opt/conda/lib/python3.12/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/opt/conda/lib/python3.12/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/opt/conda/lib/python3.12/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


,Country,Year of Treatment,All substances,Cannabis
0,Austria,2022.0,1953.0,666.0
1,Belgium,2022.0,4151.0,1850.0
2,Bulgaria,2022.0,605.0,66.0
3,Croatia,2022.0,641.0,288.0
4,Cyprus,2022.0,535.0,251.0
5,Czechia,2020.0,2519.0,539.0
6,Denmark,2022.0,3368.0,1493.0
7,Estonia,2022.0,124.0,11.0
8,Finland,2021.0,164.0,36.0
9,France,2021.0,18195.0,10680.0


**On merge ces trois tables qui vont constituer la base de notre table sur les "entrants"**
Une fois les infos globales sur les "entrants" dans chaque pays d'Europe groupées pour former une table, on rajoutera l'ensemble des données que l'on possède sur le sujet.
On commencera par effectuer des **statistiques descriptives** puis ces données serviront à entraîner notre modèle de **machin learning**.

In [11]:
### On effectue le merge (left join) afin de grouper les données par pays

## On commence par un merge "intermédiaire"
# On commence par supprimer la colonne "year of treatment" pour df 2 car cette colonne apparaît dans les deux tables
df2 = df2.drop(columns=["Year of Treatment"])
intermediaire = pd.merge(df1, df2, how="left", on="Country")
# Il faut renommer certaines colonnes (noms en doublon)
intermediaire = intermediaire.rename(columns={
    "Country": "Country",
    "Year of Treatment": "Year of Treatment",
    "All substances_x": "All substances (global)",
    "Cannabis_x": "Cannabis (global)",
    "All substances_y": "All substances (deja traite.es)",
    "Cannabis_y": "Cannabis (deja traite.es)"
})

## On reproduit les mêmes étapes mais en "mergeant" intermediaire et df3 pour finaliser cette table de BaseException
# On commence par supprimer la colonne "year of treatment" pour df 2 car cette colonne apparaît dans les deux tables
df3 = df3.drop(columns=["Year of Treatment"])
table_base = pd.merge(intermediaire, df3, how="left", on="Country")
# Il faut renommer certaines colonnes (noms en doublon)
table_base = table_base.rename(columns={
    "Country": "Country",
    "Year of Treatment": "Year of Treatment",
    "All substances (global)": "All substances (global)",
    "Cannabis (global)": "Cannabis (global)",
    "All substances (deja traite.es)": "All substances (deja traite.es)",
    "Cannabis (deja traite.es)": "Cannabis (deja traite.es)",
    "All substances": "All substances (jamais traite.es)",
    "Cannabis": "Cannabis (jamais traite.es)",
})
table_base




KeyError: "['Year of Treatment'] not found in axis"

In [ ]:
# On supprime les trois dernières lignes qui n'apportent pas d'information (lignes vides ou une seule case remplie par un "*" et une explication sur la table)
table_base = table_base.drop(index=table_base.index[-3:])
table_base # On vérifie que la dernière ligne est bien celle de la Turquie

,Country,Year of Treatment,All substances (global),Cannabis (global),All substances (deja traite.es),Cannabis (deja traite.es),All substances (jamais traite.es),Cannabis (jamais traite.es)
0,Austria,2022.0,4709.0,1124.0,2756.0,458.0,1953.0,666.0
1,Belgium,2022.0,12144.0,3501.0,7376.0,1524.0,4151.0,1850.0
2,Bulgaria,2022.0,1879.0,109.0,1164.0,42.0,605.0,66.0
3,Croatia *,2022.0,2315.0,478.0,NaN,NaN,NaN,NaN
4,Cyprus,2022.0,884.0,340.0,308.0,66.0,535.0,251.0
5,Czechia,2020.0,7283.0,1072.0,3336.0,469.0,2519.0,539.0
6,Denmark,2022.0,8241.0,3085.0,4408.0,1473.0,3368.0,1493.0
7,Estonia,2022.0,417.0,26.0,290.0,15.0,124.0,11.0
8,Finland,2021.0,427.0,62.0,263.0,26.0,164.0,36.0
9,France,2021.0,43078.0,19758.0,10628.0,3647.0,18195.0,10680.0


**Merge global**
On veut désormais *merge* toutes les tables contenant des données relatives aux soins à partir de la table *table_base*.
Pour cela, nous allons procéder en plusieurs étapes afin d'obtenir une table "propre".

- Nous allons nous supprimer les tables en doubles pour ne pas rajouter de colonnes inutiles
- On va supprimer les colonnes qui seront automatiquement en double, "Year" en l'occurrence
- On va essayer de modifier le nom des colonnes afin de faire apparaître les mention "globale", "deja traite.es" ou "jamais traite.es" et ainsi éviter que des noms de colonne dans notre table finale finissent par "_x" ou "_y" (le seul objectif ici est la lisibilité de noter code)

**ATTENTION** 
Une fois la le code de la cellule ci-dessus lancé, celui-ci ne peut plus l'être (la colonne visée n'existant donc plus).
On voit ici que je n'ai pas les bon noms de colonnes (la Liste_1 est vide), il faut donc régler cela afin que le *merge* puisse se passer sans entrave.

In [ ]:
# Etape préalable : avant d'effectuer le merge, on vérifie que la colonne 'Country' existe bien pour toutes les tables

### On commence par importer le module OS qui va nous permettre de travailler de manière globale sur les fichier plutôt que de les modifier un à un
import os

### On commence par supprimer des tables potentiellement en double
# On créer un variable correspondant au dossier contenant nos tables
dossier = "/home/onyxia/work/Projet_Python_2A/Data/EUDA/Cannabis_detaille"

# On sélectionne les fichiers excel qui sont en double
fichiers1 = [f for f in os.listdir(dossier) if f.endswith('.xlsx')] 

# On sépare en deux listes les fichier ayant et n'ayant pas de colonne nommée "Country"
Liste_1 = []
Liste_2 = []
for fichier in fichiers1:
    chemin_fichier = os.path.join(dossier, fichier)
    df = pd.read_excel(chemin_fichier)
    if "Country" in df.columns:  # Vérifie si c'est bien un fichier
        Liste_1.append(chemin_fichier)
    else : 
        Liste_2.append(chemin_fichier)
print(Liste_1, Liste_2)

        



/opt/conda/lib/python3.12/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/opt/conda/lib/python3.12/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/opt/conda/lib/python3.12/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/opt/conda/lib/python3.12/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/opt/conda/lib/python3.12/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default sty

[] ['/home/onyxia/work/Projet_Python_2A/Data/EUDA/Cannabis_detaille/sdf_jamais_traite.xlsx', '/home/onyxia/work/Projet_Python_2A/Data/EUDA/Cannabis_detaille/enfants_global.xlsx', '/home/onyxia/work/Projet_Python_2A/Data/EUDA/Cannabis_detaille/a_la_demande_de_global.xlsx', '/home/onyxia/work/Projet_Python_2A/Data/EUDA/Cannabis_detaille/age_premiere_conso_global.xlsx', '/home/onyxia/work/Projet_Python_2A/Data/EUDA/Cannabis_detaille/a_la_demande_de_deja_traite.xlsx', '/home/onyxia/work/Projet_Python_2A/Data/EUDA/Cannabis_detaille/age_premiere_conso_deja_traite.xlsx', '/home/onyxia/work/Projet_Python_2A/Data/EUDA/Cannabis_detaille/niveau_educ_deja_traite.xlsx', '/home/onyxia/work/Projet_Python_2A/Data/EUDA/Cannabis_detaille/compo_menage_femmes.xlsx', '/home/onyxia/work/Projet_Python_2A/Data/EUDA/Cannabis_detaille/niveau_educ_jamais_traite.xlsx', '/home/onyxia/work/Projet_Python_2A/Data/EUDA/Cannabis_detaille/deja_traite_entrants.xlsx', '/home/onyxia/work/Projet_Python_2A/Data/EUDA/Cannabis

/opt/conda/lib/python3.12/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/opt/conda/lib/python3.12/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/opt/conda/lib/python3.12/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/opt/conda/lib/python3.12/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/opt/conda/lib/python3.12/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default sty

On va donc faire en sorte que la ligne du tableau souhaitée (la quatrième) apparaisse comme le titre de chaque table.

In [24]:
# Etape préalable : avant d'effectuer le merge, on vérifie que la colonne 'Country' existe bien pour toutes les tables

### On commence par importer le module OS qui va nous permettre de travailler de manière globale sur les fichier plutôt que de les modifier un à un
import os

### On commence par supprimer des tables potentiellement en double
# On créer un variable correspondant au dossier contenant nos tables
dossier = "/home/onyxia/work/Projet_Python_2A/Data/EUDA/Cannabis_detaille"

# On sélectionne les fichiers excel qui sont en double
fichiers1 = [f for f in os.listdir(dossier) if f.endswith('.xlsx')] 

# On remplace les titres des colonnes par les éléments présents dans la troisième ligne de chaque dataframe
for fichier in fichiers1:
    chemin_fichier = os.path.join(dossier, fichier)
    df = pd.read_excel(chemin_fichier, header=3)
    df.to_excel(chemin_fichier, index=False) # On enregistre la modification, sinon le code ne sert à rien

test = "/home/onyxia/work/Projet_Python_2A/Data/EUDA/Cannabis_detaille/age_premier_conso_jamais_traite.xlsx"
test = pd.read_excel(test)
test #on regarde notre table test afin de vérifier que les modifications sont bien sauvegardées
    

/opt/conda/lib/python3.12/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/opt/conda/lib/python3.12/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/opt/conda/lib/python3.12/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/opt/conda/lib/python3.12/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/opt/conda/lib/python3.12/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default sty

,Country,Year of Treatment,Mean age,&lt;15,15 to 19,20 to 24,25 to 29,30 to 34,35 to 39,40 to 44,45 to 49,50 to 54,55 to 59,60 to 64,> 64,Not known / missing,Total
0,Austria,2022.0,16.0,166.0,375.0,46.0,12.0,4.0,3.0,0.0,1.0,0.0,0.0,0.0,0.0,59.0,666.0
1,Belgium,2022.0,16.0,516.0,926.0,92.0,33.0,11.0,7.0,1.0,0.0,2.0,0.0,0.0,0.0,262.0,1850.0
2,Bulgaria,2022.0,16.0,24.0,30.0,7.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,66.0
3,Croatia,2022.0,16.0,64.0,184.0,10.0,3.0,1.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,26.0,288.0
4,Cyprus,2022.0,17.0,52.0,159.0,29.0,3.0,4.0,1.0,NaN,NaN,1.0,NaN,NaN,NaN,2.0,251.0
5,Czechia,2020.0,16.0,117.0,179.0,15.0,6.0,NaN,1.0,2.0,NaN,1.0,NaN,NaN,NaN,218.0,539.0
6,Denmark,2022.0,NaN,490.0,686.0,76.0,18.0,6.0,5.0,0.0,0.0,1.0,0.0,0.0,0.0,211.0,1493.0
7,Estonia,2022.0,16.0,3.0,7.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.0
8,Finland,2021.0,16.0,7.0,23.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,36.0
9,France,2021.0,16.0,2866.0,4600.0,477.0,104.0,59.0,16.0,10.0,10.0,0.0,2.0,2.0,0.0,2434.0,10580.0


In [ ]:
### On commence par importer le module OS qui va nous permettre de travailler de manière globale sur les fichier plutôt que de les modifier un à un
import os

### On commence par supprimer des tables potentiellement en double
# On créer un variable correspondant au dossier contenant nos tables
dossier = "/home/onyxia/work/Projet_Python_2A/Data/EUDA/Cannabis_detaille"

# On sélectionne les fichiers excel qui sont en double
fichiers1 = [f for f in os.listdir(dossier) if "Copie" in f and f.endswith('.xlsx')] 

# On supprime lesdits fichiers de notre base de données pour éviter des colonnes en doublon dans notre table finale
for fichier in fichiers1:
    chemin_fichier = os.path.join(dossier, fichier)
    if os.path.isfile(chemin_fichier):  # Vérifie si c'est bien un fichier
        os.remove(chemin_fichier)  # Supprime le fichier

### On va supprimer les colonnes "Year of Treatment" présentent dans toutes les tables pour éviter les doublons dans notre table finale
# A quelques exceptions prêt négligeables, ka date est toujours 2022, l'information est donc redondante
fichiers2 = [f for f in os.listdir(dossier) if f.endswith('.xlsx')] 

for fichier in fichiers2 : 
    chemin_fichier = os.path.join(dossier, fichier)
    df = pd.read_excel(chemin_fichier)
    if "Year of Treatment" in df.columns :
        df = df.drop(columns=["Year of Treatment"])
        df.to_excel(chemin_fichier, index=False) # On enregistre la modification, sinon le code ne sert à rien

### Maintenant que ces modifications sont faites, on effectue le merge
for fichier in fichiers2 : 
    chemin_fichier = os.path.join(dossier, fichier)
    df = pd.read_excel(chemin_fichier)
    test = pd.merge(table_base, df, how="left", on="Country")
    

/opt/conda/lib/python3.12/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/opt/conda/lib/python3.12/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/opt/conda/lib/python3.12/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/opt/conda/lib/python3.12/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/opt/conda/lib/python3.12/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default sty

KeyError: 'Country'

In [7]:
import pandas as pd
import openpyxl
### On commence par les donnees generales
total = "/home/onyxia/work/Projet_Python_2A/Data/EUDA/Cannabis_detaille/drogues_entrants.xlsx"
df1 = pd.read_excel(total, header=3)
df1

/opt/conda/lib/python3.12/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


,Country,Year of Treatment,All substances,Opioids,Cocaine,Stimulants,Hypnotics and Sedatives,Hallucinogens,Volatile Inhalants,Cannabis,Other substances
0,Austria,2022.0,4709.0,1941.0,654.0,295.0,141.0,9.0,3.0,1124.0,20.0
1,Belgium,2022.0,12144.0,1897.0,3444.0,1233.0,728.0,337.0,28.0,3501.0,65.0
2,Bulgaria,2022.0,1879.0,627.0,84.0,189.0,41.0,2.0,0.0,109.0,28.0
3,Croatia *,2022.0,2315.0,1421.0,121.0,135.0,105.0,2.0,1.0,478.0,52.0
4,Cyprus,2022.0,884.0,169.0,231.0,140.0,2.0,1.0,1.0,340.0,NaN
5,Czechia,2020.0,7283.0,2747.0,62.0,3059.0,277.0,22.0,9.0,1072.0,35.0
6,Denmark,2022.0,8241.0,682.0,1397.0,360.0,131.0,13.0,11.0,3085.0,135.0
7,Estonia,2022.0,417.0,274.0,13.0,90.0,14.0,NaN,NaN,26.0,NaN
8,Finland,2021.0,427.0,167.0,3.0,138.0,57.0,0.0,0.0,62.0,0.0
9,France,2021.0,43078.0,8637.0,4174.0,962.0,1044.0,292.0,43.0,19758.0,430.0
